In [1]:
from fluidics.FluidicsProgramBuilder import FluidicProgramBuilder
from hardware.APump import APump
from hardware.HamiltonMVP import HamiltonMVP
import numpy as np
import pandas as pd
import time

program_builder = FluidicProgramBuilder()

In [2]:
program_builder.define_program()

,round,source,volume,time
0,1,SSC,2.0,6.0
1,1,B01,2.0,5.0
2,1,B01,2.0,6.0
3,1,PAUSE,5.0,5.0
4,1,READOUT WASH,2.0,9.0
5,1,IMAGING BUFFER,2.0,6.0
6,1,RUN,5.0,5.0


In [ ]:
# setup pump parameters
pump_parameters = {'pump_com_port': 'COM3',
                   'pump_ID': 30,
                   'verbose': True,
                   'simulate_pump': False,
                   'serial_verbose': False,
                   'flip_flow_direction': False}

# connect to pump
pump_controller = APump(pump_parameters)

# set pump to remote control
pump_controller.enableRemoteControl(True)

In [ ]:
# connect to valves
valve_controller = HamiltonMVP(com_port='COM4')

# initialize valves
valve_controller.autoAddress()
valve_controller.autoDetectValves()

In [ ]:
valve_controller.changePort(valve_ID=0,port_ID=0)
valve_controller.changePort(valve_ID=1,port_ID=0)
valve_controller.changePort(valve_ID=2,port_ID=0)
valve_controller.changePort(valve_ID=3,port_ID=0)

In [9]:
df_program['round'].max()

1

In [8]:
df_program = pd.read_csv('test_003.csv')
r=0

# select current round
df_current_program = df_program[(df_program['round']==r+1)]

for index, row in df_current_program.iterrows():
    # extract source name
    source_name = str(row['source'])

    # extract volume to pump and time
    pump_amount_ml = float(row['volume'])
    pump_time_min = float(row['time'])           
    
    if source_name == 'RUN':
        print('Fluidics round done, running imaging.')
    elif source_name == 'PAUSE':
        print('pausing for:' +str(pump_time_min*60)+' seconds.')
    else:
        # convert ml/min rate to pump rate
        # this is hardcoded to our fluidic setup
        # please check for your own setup
        pump_rate = -1.0

        print(np.round((pump_amount_ml/pump_time_min),2))

        if np.round((pump_amount_ml/pump_time_min),2) == 0.50:
            pump_rate = 11.0
        elif np.round((pump_amount_ml/pump_time_min),2) == 0.40:
            pump_rate = 10.0
        elif np.round((pump_amount_ml/pump_time_min),2) == 0.33:
            pump_rate = 9.0
        elif np.round((pump_amount_ml/pump_time_min),2) == 0.22:
            pump_rate = 5.0

        print('Pump setting: '+str(pump_rate))

        if pump_rate == -1.0:
            print('Error in determining pump rate. Exiting.')
            sys.exit()

0.33
Pump setting: 9.0
0.4
Pump setting: 10.0
0.33
Pump setting: 9.0
pausing for:300.0 seconds.
0.22
Pump setting: 5.0
0.33
Pump setting: 9.0
Fluidics round done, running imaging.
